In [1]:
import torch
torch.manual_seed(42)
from torch import nn
from torch.utils.data import DataLoader,Dataset

In [2]:
from privacyflow.configs import path_configs
from privacyflow.datasets import heart_dataset
from privacyflow.preprocessing import heart_preprocess
from privacyflow.models import heart_models

In [3]:
#Check if GPU is available
if torch.cuda.is_available():
    print("GPU will be used")
    device = torch.device('cpu')
else:
    print("No GPU available")
    device = torch.device('cpu')

GPU will be used


## Data Prep

Der Heart-Datensatz ist eine einzelne .csv Datei. Diese wird mittels einer Preprocessing-Methode bereinigt und in Train-Val-Test gesplittet.
Diese können anschließend als PyTorch Dataset genutzt werden.

In [4]:
from privacyflow.preprocessing import heart_preprocess
heart_preprocess.preprocess_heart_data()

        age       sex        cp  trestbps      chol       fbs   restecg  \
0  1.461912  0.668625 -0.910527  0.678354 -1.049971  2.364167  0.943787   
1 -1.236395  0.668625 -0.910527 -0.672102 -1.368875 -0.422982 -0.974987   
2  0.166725 -1.495608 -0.910527  3.829420  0.843525  2.364167 -0.974987   
3  0.382589 -1.495608 -0.910527 -1.797483  0.046264 -0.422982 -0.974987   
4  0.382589  0.668625  1.006645  0.003126 -0.432093 -0.422982 -0.974987   

    thalach     exang   oldpeak     slope        ca      thal  target  
0 -0.367005 -0.737518  2.060380 -0.648057  1.175793  1.124175       0  
1 -1.286626  1.355899  1.275912 -0.648057 -0.737625  1.124175       0  
2 -0.717337  1.355899  2.583359 -2.308704  1.175793  1.124175       0  
3 -1.199043 -0.737518 -0.031535 -0.648057 -0.737625 -0.518679       1  
4  1.034321 -0.737518  1.886054  1.012589  1.175793  1.124175       0  
        age       sex        cp  trestbps      chol       fbs   restecg  \
0  0.814318 -1.495608 -0.910527 -0.447026 

In [5]:
train_dataset = heart_dataset.HeartDataset(mode="train")
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True
)

val_dataset = heart_dataset.HeartDataset(mode="val")
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=32,
    shuffle=False
)

test_dataset = heart_dataset.HeartDataset(mode="test")
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=False
)

print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

369
3
7


## Model

In [6]:
model = heart_models.HeartModelBase(13,2)
model = model.to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [8]:
for epoch in range(10):
    model.train()
    epoch_loss = 0.0
    for batch in train_dataloader:
        inputs,labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    model.eval()
    num_correct = 0.0
    for batch in val_dataloader:
        inputs,labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        # num_correct += (outputs.round() == labels).float().sum()
        _, predicted = torch.max(outputs,1)
        num_correct += (predicted == labels).sum()
    print(f"Epoch: {epoch+1:2}, Train Loss:{epoch_loss/len(train_dataloader):.5f}, Val Acc:{num_correct/len(val_dataset):.5f}")


tensor([0, 1])
tensor([0, 0])
tensor([0, 0])
tensor([0, 1])
tensor([0, 0])
tensor([1, 0])
tensor([1, 1])
tensor([0, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1, 1])
tensor([1, 0])
tensor([0, 1])
tensor([1, 1])
tensor([0, 0])
tensor([1, 0])
tensor([1, 0])
tensor([0, 0])
tensor([1, 1])
tensor([0, 0])
tensor([0, 0])
tensor([1, 1])
tensor([1, 1])
tensor([1, 1])
tensor([1, 0])
tensor([1, 1])
tensor([0, 0])
tensor([0, 1])
tensor([1, 0])
tensor([1, 1])
tensor([0, 1])
tensor([0, 0])
tensor([1, 0])
tensor([1, 1])
tensor([0, 1])
tensor([1, 0])
tensor([1, 0])
tensor([0, 1])
tensor([0, 1])
tensor([1, 1])
tensor([1, 1])
tensor([1, 0])
tensor([0, 1])
tensor([1, 1])
tensor([0, 1])
tensor([1, 0])
tensor([1, 1])
tensor([0, 0])
tensor([1, 1])
tensor([0, 1])
tensor([0, 0])
tensor([0, 0])
tensor([0, 1])
tensor([0, 1])
tensor([1, 0])
tensor([1, 1])
tensor([1, 0])
tensor([1, 0])
tensor([1, 1])
tensor([0, 1])
tensor([1, 1])
tensor([1, 0])
tensor([1, 1])
tensor([0, 0])
tensor([1, 0])
tensor([0,

In [9]:
num_correct = 0.0
model.eval()
for batch in test_dataloader:
    inputs,labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    # num_correct += (outputs.round() == labels).float().sum()
    _, predicted = torch.max(outputs,1)
    num_correct += (predicted == labels).sum()

accuracy = num_correct/len(test_dataset)
print(accuracy)

tensor(0.9561)
